|课程名称：大数据分布式计算|学期：2023春|成绩：|
|-|-|-|
|指导教师：李青泽|学生姓名：邓力予|学生学号：20201910442|
|实验名称：MapReduce|
|学院：数学与统计学院|专业：数据科学与大数据技术|年级：2020级|

# 代码与简述

## Slave

- 我们先定义Slave需要做的工作。

In [1]:
def slave(data:str,port:int):
    from datetime import datetime
    print("【",datetime.now(),"】","Port",port,"：","Job Start")
    res=dict()
    for w in data.split():
        if w in res:
            res[w]=res[w]+1
        else:
            res[w]=1
    print("【",datetime.now(),"】","Port",port,"：","Job Done")
    return res

Slave将对收到字符串进行词频统计，在统计后将结果返回给Server

- 为了一次性启动多个Slave，可以通过一个Python脚本实现：

In [ ]:
# from threading import Thread
# from os import system



# def main(port,_):
#     system("rpyc_classic.exe -p {} -q --logfile ./slave.log".format(port))

# slaves=[]

# for i in range(8):
#     slaves.append(Thread(target=main,args=(11811+i,1)))

# for i in slaves:
#     i.start()

# for i in slaves:
#     i.join()

## Server

- 导入必要包

In [2]:
import rpyc
from collections import Counter
import os
from datetime import datetime

使用`rpyc`包进行远程RPC访问

- 读取文本文件

In [3]:
root, dirs, files=next(os.walk('./txts'))
txt_content=[]
for file in files:
    with open(root+'/'+file,'r') as txt:
        txt_content.append(txt.read())

- 创建Slave列表

In [4]:
SlavesPorts=[]
for i in range(8):
    SlavesPorts.append(11811+i)

Slave在11811~11818端口上

- Map

In [5]:
res_object=[]
SlaveIter=SlavesPorts.copy()
for __,data in enumerate(txt_content):
    while True:
        SlaveNow=SlaveIter.pop(0)
        if not SlaveIter:
                SlaveIter=SlavesPorts.copy()
        try:
            conn=rpyc.classic.connect('127.0.0.1',SlaveNow)
            func=rpyc.async_(conn.teleport(slave))
            res_object.append(func(data,SlaveNow))
            break
        except:
            continue

遍历查找可用Slave，并分配任务

- Reduce

In [6]:
print(datetime.now(),"开始统计")
res=Counter(dict())
for _,data in enumerate(res_object):
    data.wait()
    res=res+Counter(data.value)
res=dict(res)
print(datetime.now(),"结束统计")

2023-05-10 00:29:41.799562 开始统计
2023-05-10 00:30:39.958099 结束统计


- 结果写入文件

In [7]:
with open("result.txt","w") as res_file:
    res_file.write(str(dict(res)))

# 结果

- 先来看8个Slave都存活的情况

![8Slave](A:/第六学期/大数据分布式计算/DistributedComputing/MapReduce/8Slaves.png)

可见，8个Slave都成功接到任务并运行了

- 接下来模拟Slave无响应状态

将Slave启动脚本中

In [ ]:
# for i in range(8):
#     slaves.append(Thread(target=main,args=(11811+i,1)))

改为：

In [ ]:
# for i in range(5):
#     slaves.append(Thread(target=main,args=(11811+i,1)))

以此模拟3个Slave的未响应

![5Slave](A:/第六学期/大数据分布式计算/DistributedComputing/MapReduce/5Slaves.png)

可以看到监听11811`11813端口的 Slave 都获得了两次任务，整个MapReduce过程依然没有受到影响。

最后我们来看一下统计结果：

In [8]:
with open("result.txt","r") as file:
    print(file.read())

{'The': 2344, 'Project': 626, 'Gutenberg': 166, 'eBook,': 10, 'Importance': 4, 'of': 13463, 'Being': 7, 'Earnest,': 1, 'by': 2005, 'Oscar': 6, 'Wilde': 6, 'This': 305, 'eBook': 40, 'is': 3539, 'for': 4079, 'the': 29610, 'use': 200, 'anyone': 84, 'anywhere': 31, 'at': 3195, 'no': 1592, 'cost': 38, 'and': 23052, 'with': 4555, 'almost': 171, 'restrictions': 16, 'whatsoever.': 17, 'You': 774, 'may': 566, 'copy': 71, 'it,': 774, 'give': 468, 'it': 5369, 'away': 577, 'or': 1748, 're-use': 16, 'under': 371, 'terms': 165, 'License': 64, 'included': 19, 'this': 1837, 'online': 32, 'www.gutenberg.org': 4, 'Title:': 8, 'Earnest': 2, 'A': 392, 'Trivial': 2, 'Comedy': 2, 'Serious': 3, 'People': 16, 'Author:': 8, 'Release': 7, 'Date:': 9, 'August': 9, '29,': 2, '2006': 3, '[eBook': 1, '#844]': 1, 'Language:': 8, 'English': 42, 'Character': 1, 'set': 415, 'encoding:': 1, 'ISO-646-US': 1, '(US-ASCII)': 1, '***START': 1, 'OF': 108, 'THE': 188, 'PROJECT': 32, 'GUTENBERG': 24, 'EBOOK': 16, 'IMPORTANCE': 